In [1]:
from scipy import sparse
import numpy as np
import numba

In [2]:
m=4000
n=4000
density = 16*m/(m*n)
A_sparse = sparse.rand(m, n, density=density, format='coo', dtype=None, random_state=None)
A_dense = A_sparse.toarray()
A_data = A_sparse.data
A_row = A_sparse.row
A_col = A_sparse.col

x = np.random.rand(m).reshape((m,1))

In [3]:
def mult_sparse(A,x):
    return A.dot(x)

In [4]:
density

0.004

In [5]:
%timeit mult_sparse(A_sparse,x)

1000 loops, best of 3: 353 µs per loop


In [6]:
def mult_dense(A,x):
    return A @ x

In [7]:
y_dense = mult_dense(A_dense,x)
%timeit y_dense = mult_dense(A_dense,x)

100 loops, best of 3: 7.38 ms per loop


In [8]:
@numba.jit(nopython=True, cache=True)
def mult_numba(rows,cols,data,x):
    y = np.zeros(x.shape)
    it = 0
    for it in range(len(data)):
        y[rows[it],0] += data[it]*x[cols[it],0]
       
    return y


In [9]:
y_numba = mult_numba(A_row,A_col,A_data,x)
%timeit mult_numba(A_row,A_col,A_data,x)

1000 loops, best of 3: 449 µs per loop


In [10]:
@numba.vectorize([numba.float64[:](numba.int32[:], numba.int32[:], numba.float64[:], numba.float64[:])])
def mult_numbav(rows,cols,data,x):
    y = np.zeros(x.shape)
    it = 0
    for it in range(len(data)):
        y[rows[it],0] += data[it]*x[cols[it],0]
       
    return y

NotImplementedError: array(int32, 1d, A) cannot be represented as a Numpy dtype

In [ ]:
y_numba = mult_numbav(A_row,A_col,A_data,x)
%timeit mult_numbav(A_row,A_col,A_data,x)